https://www.kaggle.com/datasets/mexwell/yelp-review-dataset
This dataset is a dataset of yelp reviews and the score associated with them. 

In [1]:
!pip3 install torch transformers pandas numpy wandb scikit-learn accelerate datasets -U
import numpy as np
import pandas as pd
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 86.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 72.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 9.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━

In [3]:

import os
os.environ["WANDB_DISABLED"] = "true"
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/yelp-review-dataset/train.csv
/kaggle/input/yelp-review-dataset/test.csv


In [4]:
df = pd.read_csv("/kaggle/input/yelp-review-dataset/test.csv")
len(df)

50000

In [5]:
i = np.random.rand(len(df)) < 0.8
train_validate = df[i]
test = df[~i]
i = np.random.rand(len(train_validate)) < 0.8
train = train_validate[i]
validate = train_validate[~i]

In [6]:
from datasets import Dataset, DatasetDict

reviews = DatasetDict({
    "train": Dataset.from_pandas(train),
    "test": Dataset.from_pandas(test),
    "validation": Dataset.from_pandas(validate)
    })
reviews

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'label', 'text', '__index_level_0__'],
        num_rows: 31856
    })
    test: Dataset({
        features: ['Unnamed: 0', 'label', 'text', '__index_level_0__'],
        num_rows: 10070
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'label', 'text', '__index_level_0__'],
        num_rows: 8074
    })
})

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

num_labels = 5
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', problem_type="multi_label_classification", num_labels=num_labels)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cuda


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [8]:
def tokenise_batch(batch):
    return tokenizer(batch["text"], return_tensors='pt', padding=True, truncation=True).to(device)

def tokenise(text):
    return tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)

def make_prediction(text):
    inputs = tokenise(text)
    with torch.no_grad():
        logits = model(**inputs).logits
    logits = logits.cpu()
    prediction = torch.argmax(logits, dim=1).item()
    return prediction

In [9]:
reviews_encoded = reviews.map(tokenise_batch, batched=True)

Map:   0%|          | 0/31856 [00:00<?, ? examples/s]

Map:   0%|          | 0/10070 [00:00<?, ? examples/s]

Map:   0%|          | 0/8074 [00:00<?, ? examples/s]

In [10]:
reviews_encoded

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'label', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 31856
    })
    test: Dataset({
        features: ['Unnamed: 0', 'label', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 10070
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'label', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 8074
    })
})

In [11]:
prediction = list(map(make_prediction, reviews_encoded['test']['text']))

In [12]:
from sklearn import metrics
y_test = reviews_encoded['test']['label']
print('accuracy score: ', metrics.accuracy_score(y_test, prediction))
print(metrics.classification_report(y_test, prediction))
print("Confusion matrix:\n", metrics.confusion_matrix(y_test, prediction))

accuracy score:  0.1720953326713009
              precision    recall  f1-score   support

           0       0.10      0.14      0.12      1995
           1       0.18      0.00      0.01      2025
           2       0.20      0.71      0.31      2027
           3       0.00      0.00      0.00      1986
           4       0.00      0.00      0.00      2037

    accuracy                           0.17     10070
   macro avg       0.10      0.17      0.09     10070
weighted avg       0.10      0.17      0.09     10070

Confusion matrix:
 [[ 288    4 1703    0    0]
 [ 373    6 1646    0    0]
 [ 574   14 1439    0    0]
 [ 731    4 1251    0    0]
 [ 827    5 1205    0    0]]


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


From observation the pretrained model without any training or fine tuning is not very accurate at all only guessing one category. It mostly only guessed one category

Now we shall see what happens when we fine tune the model to our data

In [13]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

2024-04-15 05:36:32.511044: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 05:36:32.511140: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 05:36:32.634323: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [15]:
from torch.nn.functional import one_hot
def convert_one_hot(label):
    label = torch.tensor(label)
    label
    return one_hot(label, num_classes=5).to(torch.float32)

reviews_encoded_frame = reviews_encoded["train"].to_pandas()
reviews_encoded_frame["label"] = reviews_encoded_frame["label"].apply(convert_one_hot)
reviews_encoded_frame["label"] = reviews_encoded_frame["label"].apply(lambda x: x.tolist())
reviews_encoded["train"] = Dataset.from_pandas(reviews_encoded_frame)

reviews_encoded_frame = reviews_encoded["validation"].to_pandas()
reviews_encoded_frame["label"] = reviews_encoded_frame["label"].apply(convert_one_hot)
reviews_encoded_frame["label"] = reviews_encoded_frame["label"].apply(lambda x: x.tolist())
reviews_encoded["validation"] = Dataset.from_pandas(reviews_encoded_frame)


In [16]:
print(reviews_encoded['train']['label'][1])

[1.0, 0.0, 0.0, 0.0, 0.0]


In [17]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=reviews_encoded['train'],
    eval_dataset=reviews_encoded['validation'],
    tokenizer=tokenizer,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [18]:
trainer.train()

Step,Training Loss
500,0.405000
1000,0.349100
1500,0.343800
2000,0.329000
2500,0.329700
3000,0.330600
3500,0.314200
4000,0.317700
4500,0.259100
5000,0.264700


TrainOutput(global_step=11946, training_loss=0.26256524492663275, metrics={'train_runtime': 2735.1339, 'train_samples_per_second': 34.941, 'train_steps_per_second': 4.368, 'total_flos': 1.266032165511168e+16, 'train_loss': 0.26256524492663275, 'epoch': 3.0})

In [19]:
predictions = trainer.predict(reviews_encoded["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(8074, 5) (8074, 5)


In [20]:
preds = np.argmax(predictions.predictions, axis=-1)

In [21]:
y_test = reviews['validation']['label']
print('accuracy score: ', metrics.accuracy_score(y_test, preds))
print(metrics.classification_report(y_test, preds))
print("Confusion matrix:\n", metrics.confusion_matrix(y_test, preds))

accuracy score:  0.6317810255139955
              precision    recall  f1-score   support

           0       0.78      0.74      0.76      1656
           1       0.57      0.59      0.58      1625
           2       0.55      0.54      0.55      1614
           3       0.55      0.56      0.55      1629
           4       0.71      0.73      0.72      1550

    accuracy                           0.63      8074
   macro avg       0.63      0.63      0.63      8074
weighted avg       0.63      0.63      0.63      8074

Confusion matrix:
 [[1231  362   40    8   15]
 [ 292  961  328   29   15]
 [  36  316  870  357   35]
 [   8   27  293  913  388]
 [  14    8   45  357 1126]]


Just by training the model with our data set we drastically improve the model from only guessing one category to getting 60% Accuracy, which is impressive when the dataset has 5 categories. I did not change and of the training settings away from the default and it still performed well. The model was much better at guessing the extremes of the rating than guessing in the middle achieving around 20 percent more accuracy at guessing 1 star and 5 star reviews than 2 to 4 star ones.

In [23]:
del model
torch.cuda.empty_cache()
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', problem_type="multi_label_classification", num_labels=num_labels)
model.to(device)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [24]:
training_args = TrainingArguments(
    output_dir = './test-trainer',
    num_train_epochs=5,
    per_device_train_batch_size= 32,
    per_device_eval_batch_size=32,
    learning_rate = 2e-5,
    load_best_model_at_end= True,
    evaluation_strategy = "steps",
    save_strategy = 'steps'
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [25]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=reviews_encoded['train'],
    eval_dataset=reviews_encoded['validation'],
    tokenizer=tokenizer,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [27]:
torch.cuda.empty_cache()
trainer.train()

Step,Training Loss,Validation Loss
500,0.376300,0.321963
1000,0.319700,0.304385
1500,0.286000,0.301924
2000,0.279800,0.299536
2500,0.248300,0.313562
3000,0.244600,0.312386
3500,0.212300,0.324695
4000,0.212100,0.323405
4500,0.189200,0.333483


TrainOutput(global_step=4980, training_loss=0.2556927278817418, metrics={'train_runtime': 4969.594, 'train_samples_per_second': 32.051, 'train_steps_per_second': 1.002, 'total_flos': 2.11005360918528e+16, 'train_loss': 0.2556927278817418, 'epoch': 5.0})

In [28]:
predictions = trainer.predict(reviews_encoded["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(8074, 5) (8074, 5)


In [29]:
preds = np.argmax(predictions.predictions, axis=-1)

In [30]:
y_test = reviews['validation']['label']
print('accuracy score: ', metrics.accuracy_score(y_test, preds))
print(metrics.classification_report(y_test, preds))
print("Confusion matrix:\n", metrics.confusion_matrix(y_test, preds))

accuracy score:  0.6302947733465445
              precision    recall  f1-score   support

           0       0.81      0.70      0.75      1656
           1       0.57      0.60      0.59      1625
           2       0.55      0.55      0.55      1614
           3       0.55      0.55      0.55      1629
           4       0.69      0.75      0.72      1550

    accuracy                           0.63      8074
   macro avg       0.63      0.63      0.63      8074
weighted avg       0.64      0.63      0.63      8074

Confusion matrix:
 [[1165  416   52    6   17]
 [ 230  976  367   34   18]
 [  25  288  882  370   49]
 [   3   19  272  901  434]
 [   8    6   39  332 1165]]


I tried adjusting the parameters for the training to see what it would change. It seems that increasing the epochs did not allow it to learn more and it achieved the same scores as the model with the default settings whle also taking longer to train because of the increased epochs.